# Imports and Constants

In [1]:
import csv
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
NUM_CLASSES = 4
TIME_STEPS = 16 # num time steps in the point history
DIMENSION = 2 # num dimensions in the point history

# Paths

In [2]:
dataset = 'models/point_history_classifier/point_history.csv'
model_save_path = 'models/point_history_classifier/point_history_classifier.keras'

# Load Datasets

In [3]:
X_data = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (DIMENSION * TIME_STEPS) + 1)))
Y_data = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, train_size=0.8, random_state=RANDOM_SEED)

# Buid Model

In [4]:
use_lstm = False  # use LSTM (better for time series forecasting) or Dense layers (simpler feedforward network)
model = None

if use_lstm:
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(TIME_STEPS * DIMENSION, )),
        tf.keras.layers.Reshape((TIME_STEPS, DIMENSION), input_shape=(TIME_STEPS * DIMENSION, )), 
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(16, input_shape=[TIME_STEPS, DIMENSION]),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])
else:
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(TIME_STEPS * DIMENSION, )),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])

/Users/johnzhang/Desktop/Development/Projects/gesture-navigator/venv/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
I0000 00:00:1742498794.617988 3887263 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1742498794.618208 3887263 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 24)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │            44 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,086 (4.24 KB)

 Trainable params: 1,086 (4.24 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
  model_save_path,
  verbose=1,
  save_weights_only=False
)
es_callback = tf.keras.callbacks.EarlyStopping(
  patience=20, 
  verbose=1,
)

In [7]:
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)

# Train Model

In [8]:
model.fit(
  X_train,
  y_train,
  epochs=2000,
  batch_size=128,
  validation_data=(X_test, y_test),
  callbacks=[cp_callback, es_callback]
)

Epoch 1/2000
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1981 - loss: 1.3988
Epoch 1: saving model to model/point_history_classifier/point_history_classifier.keras
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.1981 - loss: 1.3988 - val_accuracy: 0.2151 - val_loss: 1.3953
Epoch 2/2000
33/34 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2071 - loss: 1.3966
Epoch 2: saving model to model/point_history_classifier/point_history_classifier.keras
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.2075 - loss: 1.3967 - val_accuracy: 0.2217 - val_loss: 1.3953
Epoch 3/2000
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2218 - loss: 1.3953
Epoch 3: saving model to model/point_history_classifier/point_history_classifier.keras
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.2216 - loss: 1.3953 - val_accuracy: 0.2075 - val_loss: 1.3939
Epoch 4/2000
31/34 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2105 - loss: 1.3987
Epoch 4: saving model to model/point_histor

# Save

In [9]:
model = tf.keras.models.load_model(model_save_path)
model.save(model_save_path, include_optimizer=False)

tflite_save_path = 'models/point_history_classifier/point_history_classifier.tflite'

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: /var/folders/xf/xbw_8nmn2t1d0kklry2h0x6c0000gn/T/tmpk2imshs4/assets


INFO:tensorflow:Assets written to: /var/folders/xf/xbw_8nmn2t1d0kklry2h0x6c0000gn/T/tmpk2imshs4/assets


Saved artifact at '/var/folders/xf/xbw_8nmn2t1d0kklry2h0x6c0000gn/T/tmpk2imshs4'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  14366226768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14366221776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14366220240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14366228112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14366227728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14366225232: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1742498839.469939 3887263 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742498839.470150 3887263 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1742498839.474422 3887263 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled


6444